In [1]:
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
us_programs = "https://ideas.repec.org/top/top.usecondept.html"
urban_programs = "https://ideas.repec.org/top/top.ure.html"
macro_programs = "https://ideas.repec.org/top/top.mac.html"

us_html = requests.get(us_programs).text
urban_html = requests.get(urban_programs).text
macro_html = requests.get(macro_programs).text

In [3]:
us_soup = BeautifulSoup(us_html, "html.parser")

school_name = []
rank = []

for row in range(5,141):
    rank.append(row-4)
    school_name.append(us_soup.find_all('tr')[row].find_all('td')[1].text)
    
us_rankings_raw = pd.DataFrame({'school_name':school_name, 'overall_rank':rank})
us_rankings = us_rankings_raw.copy()

In [4]:
urban_soup = BeautifulSoup(urban_html, "html.parser")

school_name = []
rank = []

for row in range(5,479):
    rank.append(row-4)
    school_name.append(urban_soup.find_all('tr')[row].find_all('td')[1].text)
    
urban_rankings_raw = pd.DataFrame({'school_name':school_name, 'rank_urban':rank})
urban_rankings = urban_rankings_raw.copy()
urban_rankings = urban_rankings[urban_rankings['school_name'].apply(lambda x: "(USA)" in x)]
urban_rankings = urban_rankings.reset_index()
urban_rankings['us_urban_rank'] = urban_rankings.index + 1
urban_rankings = urban_rankings.drop(columns=['index'])

In [5]:
macro_soup = BeautifulSoup(macro_html, "html.parser")

school_name = []
rank = []

for row in range(5,479):
    rank.append(row-4)
    school_name.append(macro_soup.find_all('tr')[row].find_all('td')[1].text)
    
macro_rankings_raw = pd.DataFrame({'school_name':school_name, 'rank_macro':rank})
macro_rankings = macro_rankings_raw.copy()
macro_rankings = macro_rankings[macro_rankings['school_name'].apply(lambda x: "(USA)" in x)]
macro_rankings = macro_rankings.reset_index()
macro_rankings['us_macro_rank'] = macro_rankings.index + 1
macro_rankings = macro_rankings.drop(columns=['index'])

In [6]:
merged = us_rankings.merge(urban_rankings, how='left', on='school_name')
merged['rank_diff'] = merged['overall_rank'] - merged['us_urban_rank']
merged = merged.sort_values('rank_diff', ascending=False)
merged.to_csv('../data/urban_overperformers.csv')

In [7]:
merged = us_rankings.merge(macro_rankings, how='left', on='school_name')
merged['rank_diff'] = merged['overall_rank'] - merged['us_macro_rank']
merged = merged.sort_values('rank_diff', ascending=False)
merged.to_csv('../data/macro_overperformers.csv')